## Transformer & MLP(다중신경망) 데이터 전처리
---

1. 중복컬럼 제거
    - 시군구, 시군구_수정 제거 후 '동' 컬럼 추가
    - 도로명 제거(전체주소에 포함)

  
  
  - 금액 관련 변수 수치형으로 전환
    - 종전계약 `보증금(만원), 월세금(만원)` 처리
      Nan인 경우 -1로 값을 대체 후 수치형 전환
    - 보증금, 월세금은 수치형으로 전환
  - 결측치 제거

    - 월세 범위 확인 `월세금(만원) < 0 `이 있으면 곤란
    - `전용면적`과 `전월세구분` 등 결측치 처리  
  
  ---

2. 중복 `rows & columns` 처리 및 파생변수 추가

  
  
  1. 계약날짜 생성 및 계약년월 계약일 제거
  - `df`: 모든 속성이 중복 된 행을 제거한 `데이터 프레임`
  - `df2`: 주요 속성이 중복 된 행을 제거한 `데이터 프레임`
  ---
3. 데이터 `type` 변경 및 단지별 평균가
    1. `계약기간` -> 계약 개월수 그리고 계약기간 변경
      - 계약기간 데이터 신뢰성 이슈로 이상치는 24개월로 fill
    
      ---

    2. `단지평균단가` 컬럼 추가 및 단지명 제거
      - 전월세, 주택유형 별 전월세 단지별 평균가 생성

      ---

    3. 데이터 `type` 변경
      
      - 실수형: `float: 보증금(만원), 월세금(만원), 종전계약 보증금(만원), 종전계약 월세(만원), 단지평균단가, 계약기간, 층, 전용면적(m2), index, rate, pp_rate, 단지별 평균 보증금, 단지별 평균 월세금`

        ---
      
      - 범주형: `계약구분, 갱신요구권 사용, 주택유형, 전월세구분`

        ---
      
      - datetime: `건축년도, 계약날짜`        
  ---

4. 수치형 데이터 이상치 제거 (`histogram`과 데이터 분포를 기준으로 이상치 선정)

  1. `월세금(만원)` -> 이상치 발견시 행 제거
  2. `보증금(만원)` -> 이상치 발견시 행 제거
  


#### 데이터 로드 및 확인

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 로드
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/MergedData/full_data.csv")

<ipython-input-3-798d85a3359d>:2: DtypeWarning: Columns (1,7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/MergedData/full_data.csv")


In [ ]:
df.columns

Index(['시군구', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세금(만원)',
       '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)',
       '종전계약 월세(만원)', '주택유형', 'index', 'rate', 'pp_rate', '시군구_수정', '전체주소'],
      dtype='object')

In [ ]:
df.head()

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,...,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate,시군구_수정,전체주소
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.03,2019-04,30,"18,000",0,3,2018.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 성동구,서울특별시 성동구 상원6나길 24
1,서울특별시 은평구 불광동,더클래스,전세,43.17,2019-04,30,"23,000",0,4,2019.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 은평구,서울특별시 은평구 불광로14길 4
2,서울특별시 광진구 화양동,화양타워,전세,37.34,2019-04,30,"11,000",0,4,1999.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 광진구,서울특별시 광진구 능동로19길 47
3,서울특별시 강북구 번동,효성네오인텔리안,월세,31.71,2019-04,30,"1,000",50,4,2005.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 강북구,서울특별시 강북구 도봉로 352
4,서울특별시 용산구 한강로2가,한강현대하이엘,전세,40.28,2019-04,30,"21,000",0,15,2005.0,...,-,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 용산구,서울특별시 용산구 새창로 213-12


In [ ]:
df.shape

(3194714, 22)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3194714 entries, 0 to 3194713
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   시군구           object 
 1   단지명           object 
 2   전월세구분         object 
 3   전용면적(㎡)       float64
 4   계약년월          object 
 5   계약일           int64  
 6   보증금(만원)       object 
 7   월세금(만원)       object 
 8   층             int64  
 9   건축년도          float64
 10  도로명           object 
 11  계약기간          object 
 12  계약구분          object 
 13  갱신요구권 사용      object 
 14  종전계약 보증금(만원)  object 
 15  종전계약 월세(만원)   object 
 16  주택유형          object 
 17  index         float64
 18  rate          float64
 19  pp_rate       float64
 20  시군구_수정        object 
 21  전체주소          object 
dtypes: float64(5), int64(2), object(15)
memory usage: 536.2+ MB


### 1. 중복컬럼제거 수치형 변수 전환 결측치 제거
  


  

#### 중복컬럼 제거

  - 시군구, 시군구_수정 제거 후 '동' 컬럼 추가
  - 도로명 제거(전체주소에 포함)

In [ ]:
df['동'] = df['시군구'].str.split(' ').str[-1]

In [ ]:
df

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,...,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate,시군구_수정,전체주소,동
0,서울특별시 성동구 성수동1가,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,2019-04,30,"18,000",0,3,2018.0,...,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 성동구,서울특별시 성동구 상원6나길 24,성수동1가
1,서울특별시 은평구 불광동,더클래스,전세,43.1700,2019-04,30,"23,000",0,4,2019.0,...,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 은평구,서울특별시 은평구 불광로14길 4,불광동
2,서울특별시 광진구 화양동,화양타워,전세,37.3400,2019-04,30,"11,000",0,4,1999.0,...,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 광진구,서울특별시 광진구 능동로19길 47,화양동
3,서울특별시 강북구 번동,효성네오인텔리안,월세,31.7100,2019-04,30,"1,000",50,4,2005.0,...,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 강북구,서울특별시 강북구 도봉로 352,번동
4,서울특별시 용산구 한강로2가,한강현대하이엘,전세,40.2800,2019-04,30,"21,000",0,15,2005.0,...,-,NaN,NaN,오피스텔,2203.59,1.75,5.285769,서울특별시 용산구,서울특별시 용산구 새창로 213-12,한강로2가
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194709,서울특별시 송파구 장지동,위례24단지(꿈에그린),월세,75.9200,2024-05,1,"60,900",25,5,2013.0,...,-,"60,900",0,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 위례광장로 121,장지동
3194710,서울특별시 송파구 장지동,위례신도시송파푸르지오,전세,112.1156,2024-05,1,"92,500",0,6,2015.0,...,-,NaN,NaN,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 위례광장로 185,장지동
3194711,서울특별시 송파구 잠실동,리센츠,월세,27.6800,2024-05,1,"5,000",179,19,2008.0,...,사용,"5,000",170,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 올림픽로 135,잠실동
3194712,서울특별시 송파구 잠실동,잠실엘스,전세,84.8000,2024-05,1,"100,000",0,2,2008.0,...,-,NaN,NaN,아파트,2636.52,3.50,5.091858,서울특별시 송파구,서울특별시 송파구 올림픽로 99,잠실동


In [ ]:
df.drop(columns=['시군구','시군구_수정'], inplace=True)

In [ ]:
df.drop(columns=['도로명'],inplace=True)

In [ ]:
df.columns

Index(['단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세금(만원)', '층',
       '건축년도', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)', '종전계약 월세(만원)',
       '주택유형', 'index', 'rate', 'pp_rate', '전체주소', '동'],
      dtype='object')

#### 금액 관련 변수 수치형으로 전환
  
  - 종전계약 `보증금(만원), 월세금(만원)` 처리
    Nan인 경우 -1로 값을 대체 후 수치형 전환
  - 보증금, 월세금은 수치형으로 전환


In [ ]:
df.isnull().sum()

,0
단지명,106854
전월세구분,2
전용면적(㎡),0
계약년월,0
계약일,0
보증금(만원),0
월세금(만원),0
층,0
건축년도,0
계약기간,0


In [ ]:
print(df['월세금(만원)'].isnull().sum(), df['보증금(만원)'].isnull().sum())

0 0


In [ ]:
# 수치형으로 전환
money_cols = ["보증금(만원)", "월세금(만원)", "종전계약 보증금(만원)", "종전계약 월세(만원)"]

for col in money_cols:
    df[col] = df[col].astype(str).str.replace(",", "").replace("nan", np.nan).astype(float)

df["보증금(만원)"].fillna(0, inplace=True)
df["월세금(만원)"].fillna(0, inplace=True)
df["종전계약 보증금(만원)"].fillna(-1, inplace=True)
df["종전계약 월세(만원)"].fillna(-1, inplace=True)



<ipython-input-15-8942f8782da1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["보증금(만원)"].fillna(0, inplace=True)
<ipython-input-15-8942f8782da1>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

#### 결측치 제거

  - 월세 범위 확인 `월세금(만원) < 0 `이 있으면 곤란
  - `전용면적`과 `전월세구분` 등 결측치 처리  

In [ ]:
df['월세금(만원)']

,월세금(만원)
0,0.0
1,0.0
2,0.0
3,50.0
4,0.0
...,...
3194709,25.0
3194710,0.0
3194711,179.0
3194712,0.0


In [ ]:
df[df['월세금(만원)'] < 0]

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate,전체주소,동
1317158,마곡엠밸리10단지,NaN,59.79,2022-10,13,12470.0,-2.0,12,2016.0,-,신규,-,-1.0,-1.0,아파트,2293.61,3.0,4.864415,서울특별시 강서구 마곡중앙1로 72,마곡동
2702726,마곡엠밸리10단지,NaN,59.79,2022-10,13,12470.0,-2.0,12,2016.0,-,신규,-,-1.0,-1.0,아파트,2293.61,3.0,4.864415,서울특별시 강서구 마곡중앙1로 72,마곡동


In [ ]:
df.isnull().sum()

,0
단지명,106854
전월세구분,2
전용면적(㎡),0
계약년월,0
계약일,0
보증금(만원),0
월세금(만원),0
층,0
건축년도,0
계약기간,0


In [ ]:
df.shape

(3194714, 20)

In [ ]:
df[df['전월세구분'].isnull()]

,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,index,rate,pp_rate,전체주소,동
1317158,마곡엠밸리10단지,NaN,59.79,2022-10,13,12470.0,-2.0,12,2016.0,-,신규,-,-1.0,-1.0,아파트,2293.61,3.0,4.864415,서울특별시 강서구 마곡중앙1로 72,마곡동
2702726,마곡엠밸리10단지,NaN,59.79,2022-10,13,12470.0,-2.0,12,2016.0,-,신규,-,-1.0,-1.0,아파트,2293.61,3.0,4.864415,서울특별시 강서구 마곡중앙1로 72,마곡동


In [ ]:
# 단지명과 전월세구분은 예측이 어렵기 때문에 결측이 발생한 행은 제거

df = df.dropna(subset=['단지명','전월세구분'])

### 2. 중복 `rows & columns` 처리 및 파생변수 추가

  1. `단지평균단가` 컬럼 추가 및 단지명 제거
  2. 동평균단가 컬럼 추가
  3. 계약날짜 생성 및 계약년월 계약일 제거
  - `df`: 모든 속성이 중복 된 행을 제거한 `데이터 프레임`
  - `df2`: 주요 속성이 중복 된 행을 제거한 `데이터 프레임`

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3087858 entries, 0 to 3194713
Data columns (total 20 columns):
 #   Column        Dtype  
---  ------        -----  
 0   단지명           object 
 1   전월세구분         object 
 2   전용면적(㎡)       float64
 3   계약년월          object 
 4   계약일           int64  
 5   보증금(만원)       float64
 6   월세금(만원)       float64
 7   층             int64  
 8   건축년도          float64
 9   계약기간          object 
 10  계약구분          object 
 11  갱신요구권 사용      object 
 12  종전계약 보증금(만원)  float64
 13  종전계약 월세(만원)   float64
 14  주택유형          object 
 15  index         float64
 16  rate          float64
 17  pp_rate       float64
 18  전체주소          object 
 19  동             object 
dtypes: float64(9), int64(2), object(9)
memory usage: 494.7+ MB


#### 계약날짜

In [ ]:
# 계약년월 + 계약일

df['계약날짜'] = df['계약년월'] + "-" + df['계약일'].astype(str)
df['계약날짜'] = pd.to_datetime(df['계약날짜'], format="%Y-%m-%d")


<ipython-input-23-40d5aa0cabc1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['계약날짜'] = df['계약년월'] + "-" + df['계약일'].astype(str)
<ipython-input-23-40d5aa0cabc1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['계약날짜'] = pd.to_datetime(df['계약날짜'], format="%Y-%m-%d")


In [ ]:
df.drop(columns=['계약년월','계약일'], inplace=True)

<ipython-input-24-e462d06c1a2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['계약년월','계약일'], inplace=True)


In [ ]:
# 추가변수 생성
df['계약연도'] = df['계약날짜'].dt.year  # 연도(YYYY)
df['계약월'] = df['계약날짜'].dt.month  # 월(MM)
df['계약요일'] = df['계약날짜'].dt.weekday  # 요일(월:0 ~ 일:6)
# df['계약분기'] = df['계약날짜'].dt.quarter  # 분기(1~4)


<ipython-input-25-3edca529446b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['계약연도'] = df['계약날짜'].dt.year  # 연도(YYYY)
<ipython-input-25-3edca529446b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['계약월'] = df['계약날짜'].dt.month  # 월(MM)
<ipython-input-25-3edca529446b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [ ]:
# 경과 시간
df['IMF_경과년수'] = (df['계약날짜'] - pd.Timestamp("1997-12-01")).dt.days / 365
df['금융위기_경과년수'] = (df['계약날짜'] - pd.Timestamp("2008-09-15")).dt.days / 365
# df['IMF_이후'] = (df['계약날짜'] >= pd.Timestamp("1997-12-01")).astype(int)
# df['금융위기_이후'] = (df['계약날짜'] >= pd.Timestamp("2008-09-15")).astype(int)


<ipython-input-26-cf7346d58b4e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IMF_경과년수'] = (df['계약날짜'] - pd.Timestamp("1997-12-01")).dt.days / 365
<ipython-input-26-cf7346d58b4e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['금융위기_경과년수'] = (df['계약날짜'] - pd.Timestamp("2008-09-15")).dt.days / 365


In [ ]:
df
# df = df.drop(columns=['금융위기_이후','IMF_이후'])

,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약기간,계약구분,갱신요구권 사용,...,rate,pp_rate,전체주소,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수
0,청호 서울숲 뜨레피움 오피스텔,전세,19.0300,18000.0,0.0,3,2018.0,-,-,-,...,1.75,5.285769,서울특별시 성동구 상원6나길 24,성수동1가,2019-04-30,2019,4,1,21.424658,10.627397
1,더클래스,전세,43.1700,23000.0,0.0,4,2019.0,-,-,-,...,1.75,5.285769,서울특별시 은평구 불광로14길 4,불광동,2019-04-30,2019,4,1,21.424658,10.627397
2,화양타워,전세,37.3400,11000.0,0.0,4,1999.0,-,-,-,...,1.75,5.285769,서울특별시 광진구 능동로19길 47,화양동,2019-04-30,2019,4,1,21.424658,10.627397
3,효성네오인텔리안,월세,31.7100,1000.0,50.0,4,2005.0,-,-,-,...,1.75,5.285769,서울특별시 강북구 도봉로 352,번동,2019-04-30,2019,4,1,21.424658,10.627397
4,한강현대하이엘,전세,40.2800,21000.0,0.0,15,2005.0,-,-,-,...,1.75,5.285769,서울특별시 용산구 새창로 213-12,한강로2가,2019-04-30,2019,4,1,21.424658,10.627397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194709,위례24단지(꿈에그린),월세,75.9200,60900.0,25.0,5,2013.0,202405~202605,갱신,-,...,3.50,5.091858,서울특별시 송파구 위례광장로 121,장지동,2024-05-01,2024,5,2,26.432877,15.635616
3194710,위례신도시송파푸르지오,전세,112.1156,92500.0,0.0,6,2015.0,202407~202607,신규,-,...,3.50,5.091858,서울특별시 송파구 위례광장로 185,장지동,2024-05-01,2024,5,2,26.432877,15.635616
3194711,리센츠,월세,27.6800,5000.0,179.0,19,2008.0,202406~202606,갱신,사용,...,3.50,5.091858,서울특별시 송파구 올림픽로 135,잠실동,2024-05-01,2024,5,2,26.432877,15.635616
3194712,잠실엘스,전세,84.8000,100000.0,0.0,2,2008.0,202407~202607,신규,-,...,3.50,5.091858,서울특별시 송파구 올림픽로 99,잠실동,2024-05-01,2024,5,2,26.432877,15.635616


#### 중복데이터제거

In [ ]:
df2 = df.copy()
#데이터 백업

In [ ]:
df2.shape

(3087858, 24)

In [ ]:
# 중복된 행의 인덱스 확인
duplicate_rows = df[df.duplicated( keep=False)]
print(duplicate_rows.index)  # 중복된 행의 인덱스 출력


Index([    149,     150,     175,     176,     465,     466,     674,     675,
           923,     924,
       ...
       3194704, 3194705, 3194706, 3194707, 3194708, 3194709, 3194710, 3194711,
       3194712, 3194713],
      dtype='int64', length=2567234)


In [ ]:
# 1. 중복된 행 개수 확인
print(f"전체 데이터 개수: {len(df)}")
print(f"중복된 행 개수: {df.duplicated().sum()}")  # 중복된 행 개수 출력


전체 데이터 개수: 3087858
중복된 행 개수: 1337505


In [ ]:
# 2. 중복된 데이터 그룹별 개수 확인
# 모든 열이 동일한 중복 데이터 그룹화
duplicate_counts = df[df.duplicated(keep=False)].groupby(df.columns.tolist()).size().reset_index(name='count')

# 상위 10개 출력
print(duplicate_counts.sort_values('count', ascending=False).head(10))


              단지명 전월세구분  전용면적(㎡)  보증금(만원)  월세금(만원)   층    건축년도 계약기간 계약구분  \
936605  이랜드PEER신촌    월세   17.296   5152.0     25.0  16  2021.0    -    -   
936526  이랜드PEER신촌    월세   17.296   5152.0     25.0   4  2021.0    -    -   
936587  이랜드PEER신촌    월세   17.296   5152.0     25.0  13  2021.0    -    -   
936592  이랜드PEER신촌    월세   17.296   5152.0     25.0  14  2021.0    -    -   
936555  이랜드PEER신촌    월세   17.296   5152.0     25.0   8  2021.0    -    -   
936569  이랜드PEER신촌    월세   17.296   5152.0     25.0  10  2021.0    -    -   
936533  이랜드PEER신촌    월세   17.296   5152.0     25.0   5  2021.0    -    -   
936516  이랜드PEER신촌    월세   17.296   5152.0     25.0   3  2021.0    -    -   
936541  이랜드PEER신촌    월세   17.296   5152.0     25.0   6  2021.0    -    -   
936550  이랜드PEER신촌    월세   17.296   5152.0     25.0   7  2021.0    -    -   

       갱신요구권 사용  ...   pp_rate                 전체주소    동       계약날짜  계약연도  \
936605        -  ...  4.871835  서울특별시 마포구 서강로 75-16  창전동 2020-10-21  2020   
936526   

In [ ]:
df = df.drop_duplicates(keep='first')
print(f"중복 제거 후 데이터 개수: {len(df)}")
df= df.reset_index(drop=True)

중복 제거 후 데이터 개수: 1750353


In [ ]:
df.head()

,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약기간,계약구분,갱신요구권 사용,...,rate,pp_rate,전체주소,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수
0,청호 서울숲 뜨레피움 오피스텔,전세,19.03,18000.0,0.0,3,2018.0,-,-,-,...,1.75,5.285769,서울특별시 성동구 상원6나길 24,성수동1가,2019-04-30,2019,4,1,21.424658,10.627397
1,더클래스,전세,43.17,23000.0,0.0,4,2019.0,-,-,-,...,1.75,5.285769,서울특별시 은평구 불광로14길 4,불광동,2019-04-30,2019,4,1,21.424658,10.627397
2,화양타워,전세,37.34,11000.0,0.0,4,1999.0,-,-,-,...,1.75,5.285769,서울특별시 광진구 능동로19길 47,화양동,2019-04-30,2019,4,1,21.424658,10.627397
3,효성네오인텔리안,월세,31.71,1000.0,50.0,4,2005.0,-,-,-,...,1.75,5.285769,서울특별시 강북구 도봉로 352,번동,2019-04-30,2019,4,1,21.424658,10.627397
4,한강현대하이엘,전세,40.28,21000.0,0.0,15,2005.0,-,-,-,...,1.75,5.285769,서울특별시 용산구 새창로 213-12,한강로2가,2019-04-30,2019,4,1,21.424658,10.627397


In [ ]:
df.isnull().sum()

,0
단지명,0
전월세구분,0
전용면적(㎡),0
보증금(만원),0
월세금(만원),0
층,0
건축년도,0
계약기간,0
계약구분,0
갱신요구권 사용,0


In [ ]:
df['계약기간'].value_counts()

,count
계약기간,
-,850753
202402~202602,26606
202202~202402,22987
202302~202502,22325
202303~202503,19221
...,...
202005~202201,1
202005~202208,1
202005~202010,1


In [ ]:
print(df.columns)

Index(['단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세금(만원)', '층', '건축년도', '계약기간',
       '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)', '종전계약 월세(만원)', '주택유형', 'index',
       'rate', 'pp_rate', '전체주소', '동', '계약날짜', '계약연도', '계약월', '계약요일',
       'IMF_경과년수', '금융위기_경과년수'],
      dtype='object')


In [ ]:
# 중요 컬럼만 골라서 중복 확인 (예: 계약정보, 주소, 보증금, 월세)
key_cols = ['단지명','전체주소', '전월세구분', '전용면적(㎡)', '계약날짜', "층", '보증금(만원)', '월세금(만원)']
print(f"전체 데이터 개수: {len(df2)}")
print(f"핵심 컬럼 기준 중복 개수: {df2.duplicated(subset=key_cols).sum()}")


전체 데이터 개수: 3087858
핵심 컬럼 기준 중복 개수: 1391921


In [ ]:
df2 = df2.drop_duplicates(subset=key_cols, keep='first')
print(f"중복 제거 후 데이터 개수: {len(df2)}")

중복 제거 후 데이터 개수: 1695937


In [ ]:
df2.head()

,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약기간,계약구분,갱신요구권 사용,...,rate,pp_rate,전체주소,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수
0,청호 서울숲 뜨레피움 오피스텔,전세,19.03,18000.0,0.0,3,2018.0,-,-,-,...,1.75,5.285769,서울특별시 성동구 상원6나길 24,성수동1가,2019-04-30,2019,4,1,21.424658,10.627397
1,더클래스,전세,43.17,23000.0,0.0,4,2019.0,-,-,-,...,1.75,5.285769,서울특별시 은평구 불광로14길 4,불광동,2019-04-30,2019,4,1,21.424658,10.627397
2,화양타워,전세,37.34,11000.0,0.0,4,1999.0,-,-,-,...,1.75,5.285769,서울특별시 광진구 능동로19길 47,화양동,2019-04-30,2019,4,1,21.424658,10.627397
3,효성네오인텔리안,월세,31.71,1000.0,50.0,4,2005.0,-,-,-,...,1.75,5.285769,서울특별시 강북구 도봉로 352,번동,2019-04-30,2019,4,1,21.424658,10.627397
4,한강현대하이엘,전세,40.28,21000.0,0.0,15,2005.0,-,-,-,...,1.75,5.285769,서울특별시 용산구 새창로 213-12,한강로2가,2019-04-30,2019,4,1,21.424658,10.627397


In [ ]:
df2.isnull().sum()

,0
단지명,0
전월세구분,0
전용면적(㎡),0
보증금(만원),0
월세금(만원),0
층,0
건축년도,0
계약기간,0
계약구분,0
갱신요구권 사용,0


In [ ]:
df.shape

(1750353, 24)

### 3.데이터 `type` 변경 및 단지별 평균가
  1. `계약기간` -> 계약 개월수 그리고 계약기간 변경
    - 계약기간 데이터 신뢰성 이슈로 이상치는 24개월로 fill
  
    ---

  2. `단지평균단가` 컬럼 추가 및 단지명 제거
    - 전월세, 주택유형 별 전월세 단지별 평균가 생성

    ---

  3. 데이터 `type` 변경
    
    - 실수형: `float: 보증금(만원), 월세금(만원), 종전계약 보증금(만원), 종전계약 월세(만원), 단지평균단가, 계약기간, 층, 전용면적(m2), index, rate, pp_rate, 단지별 평균 보증금, 단지별 평균 월세금`

      ---
    
    - 범주형: `계약구분, 갱신요구권 사용, 주택유형, 전월세구분`

      ---
    
    - datetime: `건축년도, 계약날짜`        
---

#### 🛠 해결책: 계약기간을 개월 수로 변환하되, 계약일 기준으로 처리

1️⃣ 계약기간(YYYYMM~YYYYMM) 형식이 올바른 경우

개월 수를 계산: (종료연도 - 시작연도) * 12 + (종료월 - 시작월)
2️⃣ 계약기간이 '-'이거나 이상한 값이면?

계약일을 기준으로 계약기간이 누락된 경우 평균 개월 수(예: 24개월)로 대체
계약 종료 월이 계약일보다 이전이면 이상치로 판단하고 제거


In [ ]:
import numpy as np

# 계약기간이 "-"인 경우 NaN으로 처리
df.loc[df['계약기간'] == "-", '계약기간'] = np.nan

# 계약기간이 있는 경우 개월 수 계산
def calculate_contract_months(period):
    if pd.isna(period):  # NaN이면 그대로 반환
        return np.nan
    try:
        start, end = period.split('~')  # 시작, 종료 분리
        start_year, start_month = int(start[:4]), int(start[4:])  # YYYY, MM 분리
        end_year, end_month = int(end[:4]), int(end[4:])  # YYYY, MM 분리

        return (end_year - start_year) * 12 + (end_month - start_month)  # 개월 수 계산
    except:
        return np.nan  # 혹시 모를 예외 처리

# 개월 수 컬럼 추가
df['계약개월수'] = df['계약기간'].apply(calculate_contract_months)




In [ ]:
df['계약개월수'].describe()

,계약개월수
count,899600.000000
mean,22.075876
std,27.646732
min,0.000000
25%,24.000000
50%,24.000000
75%,24.000000
max,21624.000000


In [ ]:
df[df['계약개월수']>100]

,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약기간,계약구분,갱신요구권 사용,...,pp_rate,전체주소,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수,계약개월수
20803,아크로펠리스,월세,18.0000,500.0,47.0,8,2004.0,201010~201910,신규,-,...,5.299121,서울특별시 구로구 경인로20나길 36,오류동,2018-10-26,2018,10,4,20.915068,10.117808,108.0
174047,동양라디안루키,전세,17.9400,14000.0,0.0,10,2004.0,202112~203012,갱신,-,...,4.753512,서울특별시 서초구 서초중앙로 49,서초동,2021-08-15,2021,8,6,23.720548,12.923288,108.0
192550,목동파라곤,월세,82.6900,5000.0,250.0,20,2004.0,202306~220506,신규,-,...,5.164801,서울특별시 양천구 목동서로 155,목동,2023-04-24,2023,4,0,25.410959,14.613699,2184.0
192553,목동파라곤,월세,82.6900,5000.0,250.0,20,2004.0,202307~220507,신규,-,...,5.164801,서울특별시 양천구 목동서로 155,목동,2023-04-24,2023,4,0,25.410959,14.613699,2184.0
196241,세담파크뷰,전세,37.7000,10000.0,0.0,4,2020.0,202304~203304,갱신,-,...,5.164801,서울특별시 강서구 가로공원로88길 42,화곡동,2023-04-02,2023,4,6,25.350685,14.553425,120.0
201509,솔리브오피스텔,전세,32.3900,11000.0,0.0,3,2004.0,202303~232503,신규,-,...,5.127733,서울특별시 금천구 두산로 28,독산동,2023-03-03,2023,3,4,25.268493,14.471233,3624.0
262767,라르플레이스2단지,월세,29.1600,1000.0,69.0,11,2019.0,201405~202505,갱신,-,...,5.162828,서울특별시 도봉구 도봉로136길 19,창동,2024-04-01,2024,4,0,26.350685,15.553425,132.0
268316,제이투 리더스뷰,월세,29.6300,1000.0,105.0,4,2022.0,202403~226003,신규,-,...,5.212908,서울특별시 강서구 곰달래로16길 34,화곡동,2024-03-07,2024,3,3,26.282192,15.484932,2832.0
273964,장안현대벤처빌,전세,43.4200,16000.0,0.0,18,2004.0,202402~203402,신규,-,...,5.241678,서울특별시 동대문구 장한로 85,장안동,2024-02-15,2024,2,3,26.224658,15.427397,120.0
282690,부강애스테이,월세,54.2100,4000.0,50.0,2,2020.0,202401~203401,신규,-,...,5.227825,서울특별시 도봉구 도봉로134길 22,창동,2024-01-08,2024,1,0,26.120548,15.323288,120.0


In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np

def convert_contract_period(row):
    try:
        if row["계약기간"] == "-":  # 계약기간이 없으면 평균값(예: 24개월) 사용
            return 24

        # YYYYMM~YYYYMM 형식인지 확인
        if "~" in row["계약기간"] and len(row["계약기간"]) == 13:
            start, end = row["계약기간"].split("~")

            # 연도 및 월 추출
            start_year, start_month = int(start[:4]), int(start[4:6])
            end_year, end_month = int(end[:4]), int(end[4:6])

            # 개월 수 계산
            contract_months = (end_year - start_year) * 12 + (end_month - start_month)

            # 계약 개월 수가 너무 크거나 작으면 이상치로 판단 (예: 0개월이거나 600개월 이상)
            if contract_months <= 0 or contract_months > 600:
                return 24  # 평균값으로 대체

            return contract_months

    except:
        pass  # 오류가 나면 기본값 반환

    return 24  # 기본값 (예: 24개월)

# 적용
df["계약개월수"] = df.apply(convert_contract_period, axis=1)


In [ ]:
df['계약개월수'].describe()

,계약개월수
count,1.750353e+06
mean,2.297539e+01
std,3.865261e+00
min,1.000000e+00
25%,2.400000e+01
50%,2.400000e+01
75%,2.400000e+01
max,6.000000e+02


In [ ]:
df['계약개월수'].count()

1750353

In [ ]:
df['계약개월수'].fillna(24, inplace=True)

<ipython-input-49-5c3f121c6e93>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['계약개월수'].fillna(24, inplace=True)


In [ ]:
df.drop(columns=["계약기간"], inplace=True)


In [ ]:
df.dtypes

,0
단지명,object
전월세구분,object
전용면적(㎡),float64
보증금(만원),float64
월세금(만원),float64
층,int64
건축년도,float64
계약구분,object
갱신요구권 사용,object
종전계약 보증금(만원),float64


In [ ]:
df

,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약구분,갱신요구권 사용,종전계약 보증금(만원),...,pp_rate,전체주소,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수,계약개월수
0,청호 서울숲 뜨레피움 오피스텔,전세,19.03,18000.0,0.0,3,2018.0,-,-,-1.0,...,5.285769,서울특별시 성동구 상원6나길 24,성수동1가,2019-04-30,2019,4,1,21.424658,10.627397,24
1,더클래스,전세,43.17,23000.0,0.0,4,2019.0,-,-,-1.0,...,5.285769,서울특별시 은평구 불광로14길 4,불광동,2019-04-30,2019,4,1,21.424658,10.627397,24
2,화양타워,전세,37.34,11000.0,0.0,4,1999.0,-,-,-1.0,...,5.285769,서울특별시 광진구 능동로19길 47,화양동,2019-04-30,2019,4,1,21.424658,10.627397,24
3,효성네오인텔리안,월세,31.71,1000.0,50.0,4,2005.0,-,-,-1.0,...,5.285769,서울특별시 강북구 도봉로 352,번동,2019-04-30,2019,4,1,21.424658,10.627397,24
4,한강현대하이엘,전세,40.28,21000.0,0.0,15,2005.0,-,-,-1.0,...,5.285769,서울특별시 용산구 새창로 213-12,한강로2가,2019-04-30,2019,4,1,21.424658,10.627397,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750348,우성아파트,월세,131.08,60000.0,110.0,8,1981.0,갱신,사용,60000.0,...,5.091858,서울특별시 송파구 올림픽로4길 42,잠실동,2024-05-01,2024,5,2,26.432877,15.635616,24
1750349,리센츠,월세,27.68,3000.0,160.0,9,2008.0,신규,-,-1.0,...,5.091858,서울특별시 송파구 올림픽로 135,잠실동,2024-05-01,2024,5,2,26.432877,15.635616,24
1750350,잠실라펜앤,월세,44.09,40000.0,30.0,2,2024.0,신규,-,-1.0,...,5.082617,서울특별시 송파구 위례성대로6길 30-9,방이동,2024-10-29,2024,10,1,26.928767,16.131507,24
1750351,잠실라펜앤,전세,29.96,35000.0,0.0,2,2024.0,신규,-,-1.0,...,5.082617,서울특별시 송파구 위례성대로6길 30-9,방이동,2024-10-22,2024,10,1,26.909589,16.112329,24


#### 갱신요구권 수정

In [ ]:
df['갱신요구권 사용'] = df['갱신요구권 사용'].map({'사용': 1, '-': 0})

In [ ]:
df['갱신요구권 사용']

,갱신요구권 사용
0,0
1,0
2,0
3,0
4,0
...,...
1750348,1
1750349,0
1750350,0
1750351,0


In [ ]:
# # 전월세 구분 원핫 인코딩
# df = pd.get_dummies(df, columns=['전월세구분'], dtype=int)



# # 계약 관련 컬럼 처리
# df = pd.get_dummies(df, columns=['계약구분'], dtype=int)

# # 도로명 주소에서 거리 데이터 추가 (API 활용 시 활용 가능)
# # df['지하철거리'], df['쇼핑센터거리'] = get_distance_from_api(df['전체주소'])

# # 단지명 처리 - 평균 전세가 기준 숫자로 변환
# df['단지별 평균 전세가'] = df.groupby('단지명')['보증금(만원)'].transform('mean')
# df.drop(columns=['단지명'], inplace=True)  # 단지명 삭제

# # 정규화 (Min-Max Scaling)
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

# # 최종 확인
# print(df.head())


#### 단지별 전월세 평균가

In [ ]:
# 전세인 경우, 단지별 + 주택유형별 평균 보증금 계산
df.loc[df['전월세구분'] == '전세', '단지별 평균 보증금'] = df[df['전월세구분'] == '전세'].groupby(['단지명', '주택유형'])['보증금(만원)'].transform('mean')

# 월세인 경우, 단지별 + 주택유형별 평균 보증금 계산
df.loc[df['전월세구분'] == '월세', '단지별 평균 보증금'] = df[df['전월세구분'] == '월세'].groupby(['단지명', '주택유형'])['보증금(만원)'].transform('mean')

# 월세인 경우, 단지별 + 주택유형별 평균 월세금 계산
df.loc[df['전월세구분'] == '월세', '단지별 평균 월세금'] = df[df['전월세구분'] == '월세'].groupby(['단지명', '주택유형'])['월세금(만원)'].transform('mean')

# 전세의 경우 월세금 컬럼을 0으로 채우기 (NaN 방지)
df['단지별 평균 월세금'] = df['단지별 평균 월세금'].fillna(0)


In [ ]:
df


,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약구분,갱신요구권 사용,종전계약 보증금(만원),...,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수,계약개월수,단지별 평균 보증금,단지별 평균 월세금
0,청호 서울숲 뜨레피움 오피스텔,전세,19.03,18000.0,0.0,3,2018.0,-,0,-1.0,...,성수동1가,2019-04-30,2019,4,1,21.424658,10.627397,24,18545.588235,0.000000
1,더클래스,전세,43.17,23000.0,0.0,4,2019.0,-,0,-1.0,...,불광동,2019-04-30,2019,4,1,21.424658,10.627397,24,22627.794118,0.000000
2,화양타워,전세,37.34,11000.0,0.0,4,1999.0,-,0,-1.0,...,화양동,2019-04-30,2019,4,1,21.424658,10.627397,24,18908.888889,0.000000
3,효성네오인텔리안,월세,31.71,1000.0,50.0,4,2005.0,-,0,-1.0,...,번동,2019-04-30,2019,4,1,21.424658,10.627397,24,655.172414,57.379310
4,한강현대하이엘,전세,40.28,21000.0,0.0,15,2005.0,-,0,-1.0,...,한강로2가,2019-04-30,2019,4,1,21.424658,10.627397,24,21936.083333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750348,우성아파트,월세,131.08,60000.0,110.0,8,1981.0,갱신,1,60000.0,...,잠실동,2024-05-01,2024,5,2,26.432877,15.635616,24,34131.420534,115.344585
1750349,리센츠,월세,27.68,3000.0,160.0,9,2008.0,신규,0,-1.0,...,잠실동,2024-05-01,2024,5,2,26.432877,15.635616,24,40770.059032,163.527155
1750350,잠실라펜앤,월세,44.09,40000.0,30.0,2,2024.0,신규,0,-1.0,...,방이동,2024-10-29,2024,10,1,26.928767,16.131507,24,40000.000000,30.000000
1750351,잠실라펜앤,전세,29.96,35000.0,0.0,2,2024.0,신규,0,-1.0,...,방이동,2024-10-22,2024,10,1,26.909589,16.112329,24,35000.000000,0.000000


In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/After_preprocessing.csv")